In [ ]:
# Initial data analysis

import pandas as pd
data = pd.read_csv('./data/Cleaned_S_P_500_Data.csv', delimiter=',', index_col=False, header=0)

import numpy as np
import pandas as pd
import networkx as nx
import graphviz
import lingam
from lingam.utils import make_dot, print_causal_directions, print_dagc
from os import listdir
from os.path import isfile, join
import statistics

In [ ]:
data = pd.read_csv('./data/Cleaned_S_P_500_Data.csv', delimiter=',', index_col=False, header=0)
predictions = pd.read_csv('./data/sp500_predictions.csv', delimiter=',', index_col=False, header=0)

predictions.columns = data.columns
print(predictions)

data_backtest = data.tail(len(predictions)+1).reset_index(drop=True)
print(data_backtest)

predicted_returns = (predictions-data_backtest[:-1])/data_backtest[:-1]
print(predicted_returns)

real_returns = data_backtest.pct_change()
print(real_returns)

In [ ]:
predicted_returns = (data.shift(-3) - data)/3
predicted_returns = predicted_returns.dropna().iloc[:-1]
predicted_returns = predicted_returns.tail(len(predictions)).reset_index(drop=True)
predicted_returns

# for i in range(len(predicted_returns)):
#     print(i)

predicted_returns.loc[4]
winners = predicted_returns.loc[4].nlargest(10).index
print(winners)
try_data = real_returns.loc[5, winners].mean()
type(try_data)

def calculate_cumulative_portfolio_returns(fraction):
    num_stocks = len(data.columns)
    num_winners = int(num_stocks * fraction)
    portfolio_returns = []

    for i in range(len(predicted_returns)):
        winners = predicted_returns.loc[i].nlargest(num_winners).index
        losers = predicted_returns.loc[i].nsmallest(num_winners).index
        winner_return = real_returns.loc[i+1, winners].mean()
        loser_return = real_returns.loc[i+1, losers].mean()
        portfolio_return = winner_return - loser_return
        portfolio_returns.append(portfolio_return)
        
    return np.exp(np.sum(np.log(np.array(portfolio_returns)+1)))-1

# returns_series = pd.Series(portfolio_returns)
# cumulative_portfolio_returns = (1 + returns_series).cumprod() - 1
calculate_cumulative_portfolio_returns(0.1)

frac = np.arange(0.01, 0.5, step=0.01)
cpr = [calculate_cumulative_portfolio_returns(f) for f in frac]

import matplotlib.pyplot as plt

plt.plot(frac, cpr)

In [ ]:
# Debugging code for networkx

import numpy as np
import pandas as pd
matrix0 = pd.read_csv('./varlingam_causal_structure/adjacencymatrix0.csv', header=None)
matrix1 = pd.read_csv('./varlingam_causal_structure/adjacencymatrix1.csv', header=None)
summary_matrix = np.abs(matrix0) + np.abs(matrix1)
summary_matrix

nonzero_columns = summary_matrix.apply(lambda row: row[row != 0].index.tolist(), axis=1)
print(nonzero_columns[0])

import networkx as nx
import numpy as np
import pandas as pd
import sys
import os
import csv
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

causal_graph_filename = "causal_graphs/sp500_graph_varlingam_lag_3.txt"
G = nx.convert_node_labels_to_integers(nx.read_adjlist(causal_graph_filename, create_using=nx.DiGraph))
causes_index = list(G.predecessors(0))
causes_index

import pandas as pd
data = pd.read_csv('./data/Cleaned_CSI_300_Data.csv', delimiter=',', index_col=False, header=0)
import numpy as np
import networkx as nx
import graphviz
import lingam
from lingam.utils import make_dot, print_causal_directions, print_dagc
from os import listdir
from os.path import isfile, join
import statistics

model = lingam.VARLiNGAM(lags=1)
model.fit(data)

summary_matrix = np.sum(np.abs(model.adjacency_matrices_), axis=0)
nonzero_columns = np.nonzero(summary_matrix[0, :])[0]
print(nonzero_columns)

causal_graph_filename = "causal_graphs/csi300_graph_varlingam_lag_1.txt"
# G = nx.convert_node_labels_to_integers(nx.read_adjlist(causal_graph_filename, create_using=nx.DiGraph))
G = nx.read_adjlist(causal_graph_filename, create_using=nx.DiGraph)
edges = list(G.edges)
# print(edges)
stock_index = 0
causes_index = list(G.predecessors(str(stock_index)))
causes_index
int_list = [int(item) for item in causes_index]
int_list

import numpy as np
import networkx as nx

summary_matrix = np.eye(30)
G_self = nx.from_numpy_array(summary_matrix.T, create_using=nx.DiGraph)
for u, v, d in G_self.edges(data=True):
    del d['weight']
G_self = nx.relabel_nodes(G_self, lambda x: str(x))
edges_self = list(G_self.edges)
print(edges_self)
stock_index = 0
causes_self_index = list(G_self.predecessors(str(stock_index)))
causes_self_index

import pandas as pd
data = pd.read_csv('./data/Cleaned_S_P_500_Data.csv', delimiter=',', index_col=False, header=0)

A = data.iloc[:, 0]
A = A.to_numpy()
A

A_lag = A[1:]
A_lag

A_lag2 = A_lag[1:]
A_lag2

X = np.column_stack((A_lag[:-1], A_lag2))
X

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X[:-1,:], A[:-3].reshape(-1, 1))
prediction = model.predict(X[-1,:].reshape(1, -1))
print(prediction)

In [ ]:
# VarLiNGAM interface prototype

import networkx as nx
import numpy as np
import pandas as pd
import os
import lingam
from lingam.utils import make_dot, print_causal_directions, print_dagc

def causal_discovery_varlingam(data, lag):
    model = lingam.VARLiNGAM(lags=lag)
    model.fit(data)
    summary_matrix = np.sum(np.abs(model.adjacency_matrices_), axis=0)
    causal_graph = nx.from_numpy_array(summary_matrix.T, create_using=nx.DiGraph)
    for u, v, d in causal_graph.edges(data=True):
            del d['weight']
    return causal_graph

def causal_discovery(data, lag_range, market_name, algorithm):
    directory = "./causal_graph"
    os.makedirs(directory, exist_ok=True)
    for i in lag_range:
        print(f"lag={i}")
        if algorithm == "varlingam":
            G = causal_discovery_varlingam(data, i)
        filename = os.path.join(directory, f'{market_name}_graph_{algorithm}_lag_{i}.txt')
        nx.write_adjlist(G, filename)

data = pd.read_csv('./data/Cleaned_S_P_500_Data.csv', delimiter=',', index_col=False, header=0)
data = data.to_numpy()

causal_discovery(data[:,:10], range(1,7), "sp500", "varlingam")

In [ ]:
# Batch test prototype

import networkx as nx
import numpy as np
import pandas as pd
import os
import lingam
from lingam.utils import make_dot, print_causal_directions, print_dagc

def causal_discovery_varlingam(data, lag):
    model = lingam.VARLiNGAM(lags=lag)
    model.fit(data)
    summary_matrix = np.sum(np.abs(model.adjacency_matrices_), axis=0)
    causal_graph = nx.from_numpy_array(summary_matrix.T, create_using=nx.DiGraph)
    for u, v, d in causal_graph.edges(data=True):
            del d['weight']
    return causal_graph

def causal_discovery(data, lag_range, market_name, algorithm):
    directory = "./causal_graph"
    os.makedirs(directory, exist_ok=True)
    for i in lag_range:
        print(f"lag={i}")
        if algorithm == "varlingam":
            G = causal_discovery_varlingam(data, i)
        filename = os.path.join(directory, f'{market_name}_graph_{algorithm}_lag_{i}.txt')
        nx.write_adjlist(G, filename)

data = pd.read_csv('./data/Cleaned_S_P_500_Data.csv', delimiter=',', index_col=False, header=0)
data = data.to_numpy()
causal_discovery(data, range(1,6), "sp500", "varlingam")


In [ ]:
# Prediction util prototype

import networkx as nx
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

def predict_single(data, lag, G, stock_index, train_frac=0.8):
    predicted = []
    causes_index = list(G.predecessors(stock_index))
    causes = data[:, causes_index]
    target = data[:, stock_index]
    Y = target[lag:]
    X = np.empty((0, lag*len(causes_index)))
    train_length = int(len(Y)*train_frac)
    
    for i in range(lag, data.shape[0]):
        lagged_vars = causes[(i-lag):i, :]
        long_vars = np.concatenate(lagged_vars)
        X = np.vstack([X, long_vars])
    
    for t in range(train_length, len(Y)):
        model = LinearRegression()
        model.fit(X[:t, :], Y[:t])
        prediction = model.predict(X[t, :].reshape(1, -1))
        predicted.append(prediction[0])
    return predicted

def predict_batch(data, lag, G, train_frac = 0.8):
    predictions = np.empty((0, 0))
    for i in range(data.shape[1]):
        print(f"Predicting stock {i}")
        predicted_list = predict_single(data, lag, G, i, train_frac)
        new_column = np.array(predicted_list).reshape(-1, 1)
        predictions = np.hstack((predictions, new_column)) if predictions.size else new_column
    return predictions

def calculate_cumulative_portfolio_returns(data, predictions, winner_frac):
    data_backtest = data[-(predictions.shape[0]+1):]
    predicted_returns = (predictions - data_backtest[:-1]) / data_backtest[:-1]
    real_returns = (data_backtest[1:] - data_backtest[:-1]) / data_backtest[:-1]
    num_stocks = data.shape[1]
    num_winners = int(num_stocks * winner_frac)
    portfolio_returns = []
    winners = np.argpartition(predicted_returns, -num_winners, axis=1)[:, -num_winners:]
    losers = np.argpartition(predicted_returns, num_winners, axis=1)[:, :num_winners]

    for i in range(predictions.shape[0]):
        winner_return = np.mean(real_returns[i, winners[i]])
        loser_return = np.mean(real_returns[i, losers[i]])
        portfolio_return = winner_return - loser_return
        portfolio_returns.append(portfolio_return)
        
    return np.exp(np.sum(np.log(np.array(portfolio_returns)+1)))-1

# predict
G = nx.convert_node_labels_to_integers(nx.read_adjlist('./causal_graph/sp500_graph_varlingam_lag_3.txt', create_using=nx.DiGraph))
data = pd.read_csv('./data/Cleaned_S_P_500_Data.csv', delimiter=',', index_col=False, header=0)
column_names = data.columns.tolist()
data = data.to_numpy()
predictions = predict_batch(data, 3, G)
np.savetxt("./predictions/sp500_predictions_varlingam_lag_3.csv", predictions, delimiter=",", header=",".join(column_names), comments='')

# backtest
import matplotlib.pyplot as plt

frac = np.arange(0.01, 0.5, step=0.01)
cpr = [calculate_cumulative_portfolio_returns(data, predictions, f) for f in frac]

plt.plot(frac, cpr)